In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Rescaling, Conv2D, MaxPool2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [2]:
BATCH_SIZE = 16
IMG_HEIGHT = 224
IMG_WIDTH = 224
DATA_DIR = 'data'
EPOCHS = 10

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 6632 files belonging to 3 classes.
Using 5969 files for training.
Metal device set to: Apple M1


2022-01-31 12:03:23.188229: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-31 12:03:23.188565: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 6632 files belonging to 3 classes.
Using 663 files for validation.


In [4]:
class_names = train_ds.class_names
print(class_names)

['Igneous', 'Metamorphic', 'Sedimentary']


In [16]:
resnet50 = tf.keras.applications.resnet_v2.ResNet50V2(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

In [7]:
# model = keras.Sequential([
#     Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
#     resnet50,
#     Flatten(),
#     Dense(1024, activation='relu'),
#     Dense(256, activation='relu'),
#     Dense(3, activation='softmax')
# ])
model = keras.Sequential(
    [
     Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
     Conv2D(128, (3, 3), activation="relu"),
     MaxPool2D(),
     Conv2D(64, (3, 3), activation="relu"),
     MaxPool2D(),
     Conv2D(64, (3, 3), activation="relu"),
     MaxPool2D(),
     Conv2D(32, (3, 3), activation="relu"),
     MaxPool2D(),
     Flatten(),
     Dense(256),
     Dense(128),
     Dense(3, activation='softmax')
    ]
)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        3

In [9]:
model.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

Epoch 1/10


2022-01-31 12:03:54.511679: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-31 12:03:54.512332: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


231/374 [=================>............] - ETA: 45s - loss: 1.1050 - accuracy: 0.3585